In [2]:
import os
import shutil
import random
import pandas as pd
import sqlite3
from bson.objectid import ObjectId

import utils

from database import db, mongodb, db_sqlite

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def format_doc(doc):
    doc['id'] = str(doc['_id'])
    doc.pop('_id')
    return doc

In [4]:
def datasets():
    co = db.dataset
    docs = co.find()
    res = []
    for doc in docs:
        res.append(format_doc(doc))
    return res
datasets()

[{'data': 'data_5a260aeab037cbb588eafc87',
  'id': '5a260aeab037cbb588eafc87',
  'label2id': {'AB': 0, 'N': 1},
  'labels': ['AB', 'N'],
  'models': [1, 2, 3, 4],
  'name': 'eyes'},
 {'data': 'data_5a260aeab037cbb588eafc88',
  'id': '5a260aeab037cbb588eafc88',
  'label2id': {'MS': 2, 'N': 3, 'PD': 1, 'PS': 0},
  'labels': ['PS', 'PD', 'MS', 'N'],
  'models': [5, 6, 7],
  'name': 'brain'}]

In [5]:
def dataset_by_id(dataset_id):
    co = db.dataset
    doc = co.find_one({'_id':ObjectId(dataset_id)})
    doc = format_doc(doc)
    doc['models'] = list(set(doc['models']))
    return doc
#dataset_by_id('5a2496c6b037cbff714fc2f5')

In [6]:
def dataset_by_name(dataset_name):
    co = db.dataset
    doc = co.find_one({'name':dataset_name})
    doc = format_doc(doc)
    doc['models'] = list(set(doc['models']))
    return doc
dataset_by_name('eyes')

{'data': 'data_5a260aeab037cbb588eafc87',
 'id': '5a260aeab037cbb588eafc87',
 'label2id': {'AB': 0, 'N': 1},
 'labels': ['AB', 'N'],
 'models': [1, 2, 3, 4],
 'name': 'eyes'}

In [7]:
def data_df_by_dataset_id(dataset_id):
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from data_{}
        '''.format(dataset_id)
        df = pd.io.sql.read_sql(q, con)
    return df
#data_df_by_dataset_id('5a2496c6b037cbff714fc2f5')

In [8]:
def data_df_by_dataset_name(dataset_name):
    data = dataset_by_name(dataset_name)['data']
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from {}
        '''.format(data)
        df = pd.io.sql.read_sql(q, con)
    return df
#data_df_by_dataset_name('eyes')

In [9]:
def models():
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from model
        '''
        df = pd.io.sql.read_sql(q,con)
    return df
models()

,id,name,validation_table,dataset_id,dataset_name
0,1,eyes_lenet1,validation_5a260ba0b037cbb5ae741317,5a260aeab037cbb588eafc87,eyes
1,2,eyes_lenet2,validation_5a260d02b037cbb6d90c3dcd,5a260aeab037cbb588eafc87,eyes
2,3,eyes_lenet3,validation_5a260dceb037cbb7ca64be27,5a260aeab037cbb588eafc87,eyes
3,4,eyes_lenet4,validation_5a260e88b037cbb8ada2d955,5a260aeab037cbb588eafc87,eyes
4,5,brain_lenet1,validation_5a260ebfb037cbb9b1df6d11,5a260aeab037cbb588eafc88,brain
5,6,brain_lenet2,validation_5a260ef4b037cbba424037b0,5a260aeab037cbb588eafc88,brain
6,7,brain_lenet3,validation_5a260f29b037cbbacf7a709d,5a260aeab037cbb588eafc88,brain


In [10]:
def model_by_id(id):
    id = int(id)
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from model where id={}
        '''.format(id)
        df = pd.io.sql.read_sql(q,con)
        model_info = df.T.to_dict()[0]
    return model_info
#model_by_id(1)

In [11]:
def validation_df(key):
    key = str(key)
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from {}
        '''.format(key)
        df = pd.io.sql.read_sql(q,con)
    return df
#validation_table('validation_5a249051b037cbfa20b6e074')

In [12]:
def scores(model_id):
    model_info = model_by_id(model_id)
    df = validation_df(model_info['validation_table'])
    #dataset_info = dataset_by_id(model_info['dataset_id'])
    dataset_name = model_info['dataset_name']
    dataset_info = dataset_by_name(dataset_name)

    n_label = len(dataset_info['labels'])
    columns = ['data_id', 'fname', 'path', 'label', 'true', 'pred', 'pred_label', 'match']
    ds = list(df[columns].T.to_dict().values())
    
    y_preds = df[[str(i) for i in range(n_label)]].as_matrix().tolist()
    
    for d, y_pred in zip(ds, y_preds):
        d.update({
            'accuracy': df['match'].sum()/df['match'].count(),
            'y_pred': y_pred
        })
    res = ds
    return res
#scores(2)

[{'accuracy': 0.625,
  'data_id': 2157,
  'fname': '108_0075.JPG',
  'label': 'N',
  'match': 1,
  'path': './dataset/eyes/N/108_0075.JPG',
  'pred': 1,
  'pred_label': 'N',
  'true': 1,
  'y_pred': [0.0, 1.0]},
 {'accuracy': 0.625,
  'data_id': 1875,
  'fname': '182_0016.JPG',
  'label': 'N',
  'match': 1,
  'path': './dataset/eyes/N/182_0016.JPG',
  'pred': 1,
  'pred_label': 'N',
  'true': 1,
  'y_pred': [4.833043567487039e-05, 0.9999517202377319]},
 {'accuracy': 0.625,
  'data_id': 529,
  'fname': '129_0117.JPG',
  'label': 'N',
  'match': 1,
  'path': './dataset/eyes/N/129_0117.JPG',
  'pred': 1,
  'pred_label': 'N',
  'true': 1,
  'y_pred': [0.0, 1.0]},
 {'accuracy': 0.625,
  'data_id': 853,
  'fname': '185_0037.JPG',
  'label': 'N',
  'match': 1,
  'path': './dataset/eyes/N/185_0037.JPG',
  'pred': 1,
  'pred_label': 'N',
  'true': 1,
  'y_pred': [0.0, 1.0]},
 {'accuracy': 0.625,
  'data_id': 1538,
  'fname': '173_0023.JPG',
  'label': 'N',
  'match': 0,
  'path': './dataset/eye

In [13]:
def upload_data(data_name, data_path):
    with sqlite3.connect(db_sqlite) as con:
        c = con.cursor()
        q = '''
        insert into upload_data(name, path) values(?,?)
        '''
        v = (data_name, data_path)
        c.execute(q,v)
        data_id = c.lastrowid
        con.commit()
    return data_id

#upload_data('test2', 'test/path2')

In [4]:
def uploads_df():
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from upload_data
        '''
        df = pd.io.sql.read_sql(q,con)
    return df
uploads_df()

,id,name,path
0,1,105_0018.JPG,uploads/105_0018.JPG


In [15]:
def uploaded_data(data_id):
    with sqlite3.connect(db_sqlite) as con:
        q = '''
        select * from upload_data where id={}
        '''.format(data_id)
        df = pd.io.sql.read_sql(q,con)
        res = list(df.T.to_dict().values())[0]
    return res
try:
    uploaded_data(1)
except:
    pass